In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import os,random, math, psutil, pickle

import pandas_profiling as pdp

from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

import category_encoders as ce


In [2]:
train = pd.read_csv('./input/bank_marketing_train.csv')

In [3]:
##渾身の特徴量エンジニアリングの名残
education_rank = {
    "unknown": 0,
    "illiterate": 1,
    "basic.4y" : 2,
    "basic.6y"  :3,
    "basic.9y":4,
    "high.school":5, 
    "professional.course":6 ,
    "university.degree":7
}

# 前処理

In [4]:
#numeric = [ 'cons.price.idx', 'cons.conf.idx', 'age','campaign','pdays', 'previous'  ]
numeric = ['age','campaign','pdays', 'previous'  ]
categoricals = ['job', 'marital',  'default', 'housing', 'loan',
#       'contact', 'day_of_week',  'poutcome', 'emp.var.rate', 'euribor3m', 'nr.employed']
     # 'contact', 'day_of_week',  'poutcome', 'euribor3m']
      'education','contact', 'day_of_week',  'poutcome']#, 'euribor3m']

rank_categoricals = ["education"]
feat_cols = numeric + categoricals


In [5]:
def preprocessing ( df ) :
    df.drop(['duration'], axis=1, inplace=True)
    df["y"].replace("no",0, inplace=True)
    df["y"].replace("yes",1, inplace=True)
    # ダミー化
    ce_oe = ce.OrdinalEncoder(cols=categoricals,handle_unknown='impute')
    df = ce_oe.fit_transform(df)
    return df


In [6]:
# カスタムmetric

def exp_val_func (y_pred , data):
    valid_y = data.get_label().tolist()
    tmp_exp_val = 0
    
    y_pred = y_pred.round(0).astype("u1") 
    for i in range (len(valid_y)) :
        if y_pred[i]:
            tmp_exp_val -= 500
            if valid_y[i]:
                tmp_exp_val += 5000
    
    return 'exp_val', tmp_exp_val, True

In [7]:
train = preprocessing(train)

# モデル作成

In [8]:
train.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'day_of_week', 'campaign', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y'],
      dtype='object')

# ポイント

In [9]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': {'None'}, # custom metricにするため
            'subsample': 0.8, # 汎化性
            'subsample_freq': 1,
            'learning_rate': 0.01, # 0.1で試して、0.01で最終
            'num_leaves': 14, # 2^3 ~ 2^6
            'feature_fraction': 0.9,
            'lambda_l1': 2,  # 汎化性
            'lambda_l2': 2,  # 汎化性
            'min_data_in_leaf': 20, # 汎化性
            'scale_pos_weight':9 # 重みづけ
            #'device':'gpu'
            }

folds = 5
seed = 666

kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

In [10]:

i =0
for train_index, val_index in kf.split(train, train['y']):
    train_X = train[feat_cols].iloc[train_index] 
    train_y = train["y"].iloc[train_index] 
    val_X = train[feat_cols].iloc[val_index]
    val_y = train["y"].iloc[val_index]
    i = i+1
    break

In [11]:
## オーバーサンプリングやめました！！


# from imblearn.over_sampling import SMOTE

# train_X_origin = train_X.copy()
# train_y_origin = train_y.copy()

# sm = SMOTE(random_state=42)
# train_X, train_y = sm.fit_sample(train_X, train_y)

# p = list(zip(train_X, train_y))
# random.shuffle(p)
# train_X, train_y = zip(*p)


In [12]:
lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=categoricals)
lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=categoricals)

In [13]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=100,
                verbose_eval = 100,
                
                feval = exp_val_func,
               )

Training until validation scores don't improve for 100 rounds.
[100]	training's exp_val: 3.643e+06	valid_1's exp_val: 808000
[200]	training's exp_val: 5.339e+06	valid_1's exp_val: 1.105e+06
[300]	training's exp_val: 5.6605e+06	valid_1's exp_val: 1.16e+06
[400]	training's exp_val: 5.829e+06	valid_1's exp_val: 1.1595e+06
Early stopping, best iteration is:
[344]	training's exp_val: 5.755e+06	valid_1's exp_val: 1.1835e+06


In [14]:
fi = pd.DataFrame(  {0: train_X.columns ,  1: gbm.feature_importance()})
fi

,0,1
0,age,1123
1,campaign,602
2,pdays,270
3,previous,102
4,job,655
5,marital,214
6,default,240
7,housing,144
8,loan,105
9,education,438


In [15]:
from sklearn.metrics import confusion_matrix
# y_true = [0, 0, 0, 0, 1, 1, 1, 0, 1, 0]   # 実際の値 (0:スパムでない, 1:スパム)
# y_pred = [0, 0, 0, 0, 1, 1, 1, 1, 0, 1]   # 識別結果 (0:スパムでない, 1:スパム)
y_pred = gbm.predict(val_X)


In [16]:
#y_pred.tolist()

In [17]:
val_y_c = val_y.tolist()
y_pred_c =  y_pred.tolist()


# ポイント

In [18]:
# labels = ["not cvr", "cvr"]
#confusion_matrix( val_y_c, y_pred.round(), labels=labels, index=labels)

from pycm import ConfusionMatrix
cm = ConfusionMatrix(actual_vector=val_y_c, predict_vector=y_pred.round(0).astype("u1") )  
cm.print_matrix()

Predict    0          1          
Actual
0          4070       1962       

1          236        481        




In [19]:
test = pd.read_csv('./input/bank_marketing_test.csv')

In [20]:
test = preprocessing(test)

In [21]:
test_X = test[feat_cols]
#test_X

In [22]:

test_y = np.array(test["y"])
test_y

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
y_test_pred = gbm.predict(test_X)

In [24]:
test_y_c = test_y.tolist()
y_test_pred_c =  y_test_pred.tolist()

In [25]:
from pycm import ConfusionMatrix
cm_t = ConfusionMatrix(actual_vector=test_y_c, predict_vector=y_test_pred.round(0).astype("u1") )  
cm_t.print_matrix()

Predict    0          1          
Actual
0          4901       2399       

1          308        629        


